In [1]:
import pandas as pd
import numpy as np  
import os

In [2]:
def process_duplicates(df):
    duplicate_players = df[df.duplicated('Player', keep=False)]
    tot_df = duplicate_players[duplicate_players['Tm'] == 'TOT']
    filtered_duplicates = duplicate_players[~duplicate_players['Player'].isin(tot_df['Player'])]
    unique_non_tot = filtered_duplicates.drop_duplicates('Player', keep=False)
    final_df = pd.concat([df[~df['Player'].isin(duplicate_players['Player'])], tot_df, unique_non_tot]).reset_index(drop=True)
    
    return final_df


In [3]:
base_path = "per_100_poss"

season_data_frames = []
for year in range(1999, 2023):
    #getting the directory of each file 
    season_file_name = f"{str(year)[2:]}-{str(year+1)[2:]}nba.csv"
    season_path = os.path.join(base_path, season_file_name)
    
    if os.path.isfile(season_path):
        season_data = pd.read_csv(season_path)
        season_data.drop(columns=['Rk', 'Player-additional'], errors='ignore', inplace=True)
        season_data['Season'] = f"{year}-{year+1}" 
        
        season_data = process_duplicates(season_data) # if a team was on multiple teams, they will be dropped, 
        
        season_data_frames.append(season_data)
    else:
        print(f"File not found for season: {year}-{year+1}")
combined_df = pd.concat(season_data_frames, ignore_index=True)

combined_df

combined_df.to_csv("combined/99-23NBA_per_100_poss.csv")